<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Library
# Example: Explore ICOS Stations

## Documentation
Full documentation for the library on the [project page](https://icos-carbon-portal.github.io/pylib/), how to install and wheel on [pypi.org](https://pypi.org/project/icoscp/), source available on [github](https://github.com/ICOS-Carbon-Portal/pylib).

## Import the library

In [ ]:
from icoscp.station import station

## Get Help

In [ ]:
help(station)

## List of stations
Get a list of all ICOS stations as a pandas dataframe table with basic information. Only final certified ICOS stations are listed.
<br>Good to know:
- uri => landing page for the station at the ICOS Carbon Portal
- id  => ICOS internal station id
- elevation => meter above sea level
- theme => ICOS internal domain mapping ES=EcoSystem, AS=Atmosphere, OS=Ocean


In [ ]:
station.getIdList()

## Map of stations
Generate a folium map of stations. Each marker in the map represents a station and contains station related information.

In [ ]:
station.getIdList(project='ICOS', outfmt='map')

## Station meta data
Extract a station and display corresponding info, data products and a list of data objects

In [ ]:
myStation = station.get('ZEP')
myStation.info()

### Products
List all products (data object specifictation) associated to a specifif station

In [ ]:
myStation.products()

### Data
List all data object associated to a specific station.You can use the keyword `level=` for example to see only level 2 data objects. If you omit the keyword, all are returned.

In [ ]:
myStation.data(level=2)

## Station data object identifiers
Extract all level 2 data object identifiers. This is the id (PID) you need to extract the actual values (data), as shown in the example 1: [ex1_data](./ex1_data.ipynb)

In [ ]:
myStation.data(2)['dobj'].values